In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

In [15]:
class InceptionBlock(nn.Module):
    def __init__(self,in_channels,out_1x1,red_3x3,out_3x3,red_5x5,out_5x5,out_pool):
        super().__init__()

        # Branch 1 - 1x1 block
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, out_1x1,kernel_size = 1),
            nn.BatchNorm2d(out_1x1),
            nn.ReLU(inplace = True)
        )

        # Branch 2 - 1x1 reduction to 3x3
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, red_3x3, kernel_size = 1),
            nn.BatchNorm2d(red_3x3),
            nn.ReLU(True),
            nn.Conv2d(red_3x3, out_3x3, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(out_3x3),
            nn.ReLU(True)
        )

        # Branch 3 - 1x1 reduction to 5x5
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels,red_5x5, kernel_size=1),
            nn.BatchNorm2d(red_5x5),
            nn.ReLU(True),
            nn.Conv2d(red_5x5, out_5x5, kernel_size =5, padding = 2),
            nn.BatchNorm2d(out_5x5),
            nn.ReLU(True)
        )

        # Branch 4 - MaxPool to 1x1 conv
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride = 1, padding = 1),
            nn.Conv2d(in_channels, out_pool,kernel_size = 1),
            nn.BatchNorm2d(out_pool),
            nn.ReLU(True)
        )
    def forward(self,x):
        # concatenate all the branches
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        b4 = self.branch4(x)
        return torch.concat([b1,b2,b3,b4],1)
        

In [16]:

class MiniGoogLeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(MiniGoogLeNet, self).__init__()
        
        # 1. PRE-STEM
        # Initial Conv layer to get meaningful features before Inception blocks
        self.pre_layers = nn.Sequential(
            nn.Conv2d(3, 192, kernel_size=3, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(True),
        )

        # 2. STAGE 1 (Resolution 32x32)
        # Block A: in=192 -> out=64+128+32+32 = 256
        self.a3 = InceptionBlock(192,  64,  96, 128, 16, 32, 32)
        # Block B: in=256 -> out=128+192+96+64 = 480
        self.b3 = InceptionBlock(256, 128, 128, 192, 32, 96, 64)

        # DOWNSAMPLE 1
        self.maxpool3 = nn.MaxPool2d(3, stride=2, padding=1)

        # 3. STAGE 2 (Resolution 16x16)
        # Block C: in=480 -> out=192+208+48+64 = 512
        self.a4 = InceptionBlock(480, 192,  96, 208, 16, 48, 64)
        # Block D: in=512 -> out=160+224+64+64 = 512
        self.b4 = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        # Block E: in=512 -> out=128+256+64+64 = 512
        self.c4 = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        
        # DOWNSAMPLE 2
        self.maxpool4 = nn.MaxPool2d(3, stride=2, padding=1)

        # 4. STAGE 3 (Resolution 8x8)
        # Block F: in=512 -> out=256+320+128+128 = 832
        self.a5 = InceptionBlock(512, 256, 160, 320, 32, 128, 128)
        # Block G: in=832 -> out=384+384+128+128 = 1024
        self.b5 = InceptionBlock(832, 384, 192, 384, 48, 128, 128)

        # 5. CLASSIFIER
        # Global Average Pooling averages each 8x8 feature map into a single number
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.linear = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.pre_layers(x)
        
        x = self.a3(x)
        x = self.b3(x)
        x = self.maxpool3(x)
        
        x = self.a4(x)
        x = self.b4(x)
        x = self.c4(x)
        x = self.maxpool4(x)
        
        x = self.a5(x)
        x = self.b5(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.linear(x)
        return x

In [17]:
def train_CIFAR(architecture_class,optimizer_name = 'Adam',epochs = 10):
    # Data loading and transformation

    transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.4914, 0.4822, 0.4465),
            std=(0.2470, 0.2435, 0.2616)
        )
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.4914, 0.4822, 0.4465),
            std=(0.2470, 0.2435, 0.2616)
        )
    ])

    train_dataset = datasets.CIFAR10(root="./data-CIFAR",download = True, train=True, transform=transform_train)
    test_dataset  = datasets.CIFAR10(root="./data-CIFAR",download = True, train=False, transform=transform_test)

    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    test_loader  = DataLoader(test_dataset, batch_size=128, shuffle=False)

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Optimizer Part

    model = architecture_class().to(device)
    criterion = nn.CrossEntropyLoss()
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
    elif optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    else:
        print("Unknown optimizer! Defaulting to Adam.")
        optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # ------- training loop --------

    epochs = epochs

    for epoch in range(epochs):
        model.train()
        correct = 0
        total = 0
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()

        print(f"Epoch [{epoch+1}/{epochs}] "
            f"Loss: {running_loss/len(train_loader):.4f} "
            f"Train Acc: {100*correct/total:.2f}%")
    
    # Test Evaluation Part

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()

    print(f"Test Accuracy: {100*correct/total:.2f}%")



In [18]:
train_CIFAR(architecture_class=MiniGoogLeNet,optimizer_name='Adam')

Epoch [1/10] Loss: 1.1709 Train Acc: 57.60%
Epoch [2/10] Loss: 0.7038 Train Acc: 75.49%
Epoch [3/10] Loss: 0.5410 Train Acc: 81.26%
Epoch [4/10] Loss: 0.4492 Train Acc: 84.43%
Epoch [5/10] Loss: 0.3825 Train Acc: 86.94%
Epoch [6/10] Loss: 0.3409 Train Acc: 88.20%
Epoch [7/10] Loss: 0.2974 Train Acc: 89.67%
Epoch [8/10] Loss: 0.2661 Train Acc: 90.83%
Epoch [9/10] Loss: 0.2405 Train Acc: 91.68%
Epoch [10/10] Loss: 0.2195 Train Acc: 92.49%
Test Accuracy: 87.67%
